<a href="https://colab.research.google.com/github/k1151msarandega/QuCode-21-Days-of-Quantum-Challenge-Diary/blob/main/Day12_Quantum_Measurement_%26_No_Cloning_Theorem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 12 — Quantum Measurement & No-Cloning Theorem

> _QuCode 21 Days of Quantum Challenge — Learning notebook_
>
> **Date:** 2025-09-03  
> **Author:** YOUR_NAME  
> **Tags:** quantum, learning, challenge, day-12
>
> **Learning objectives**
> - Explain projective (von Neumann) measurement and state update (collapse/projection).
> - Compute post-measurement states and probabilities for partial measurements on multi-qubit systems.
> - State and prove the no-cloning theorem (linearity proof and inner-product preservation proof).
> - Recognise why cloning is allowed for orthogonal bases but not for arbitrary unknown states.
> - Connect measurement collapse examples to the uncertainty principle (position vs momentum intuition).
>
> **Key takeaways (summary-first)**
> - Measurement outcomes are eigenvalues; states collapse to the corresponding eigenstates and must be **renormalised**.
> - For composite systems, **partial measurements** project onto subspaces consistent with the observed outcomes.
> - **No-cloning theorem:** no unitary can clone an arbitrary, unknown state; linearity and inner-product preservation are violated.
> - Cloning *is* possible for a **fixed orthonormal set** (e.g., computational basis) via controlled copy, but not for arbitrary superpositions.
> - Measuring one observable (e.g., momentum) generally **destroys information** about a non-commuting one (e.g., position).


## Resources
- **Official/Assigned:**
    - [*Advanced Maths: Quantum Computing #9 — Two Proofs of No-Cloning Theorem*](https://www.youtube.com/watch?v=qtjdyAziXr8)  
    - [*Let's Code Physics — Measurement and wave function collapse (Computational Quantum Mechanics 3)*](https://www.youtube.com/watch?v=QmZJf3ipW7g)  
    - [*Armin Rahmani — Projective measurements in quantum computing*](https://www.youtube.com/watch?v=VxoMYQIzA6k)  
    - [*Science Discussed — Do Quantum Wavefunctions Actually Collapse?*](https://www.youtube.com/watch?v=1pLynSfLucs)
- **Extra reading:**

- **Original notes:**


In [ ]:
# %% [markdown]
# ### Environment setup (Colab)
# If you are running on Colab for the first time today, uncomment to install.
# This cell intentionally avoids heavy installs by default.
#
# !pip -q install qiskit pennylane matplotlib numpy

import sys, platform, math, json, numpy as np

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
np.random.seed(42)


## 1. Concepts in brief

### Projective measurement (single system)
- Observable $M = \sum_m m\,\Pi_m$ with projectors $\{\Pi_m\}.$  
- For a state $|\psi\rangle$:  
  - Outcome $m$ occurs with probability $p(m) = \langle\psi|\Pi_m|\psi\rangle.$  
  - Post-measurement state (Lüders rule): $|\psi'\rangle = \Pi_m|\psi\rangle / \sqrt{p(m)}.$

### Partial measurement (multipartite)
- If you measure subsystem $A$ in a joint state $\rho_{AB}$ and get $m,$ the (unnormalised) post-measurement state is  
  $(\Pi_m^A \otimes I_B)\,\rho_{AB}\,(\Pi_m^A \otimes I_B).$ Renormalise by its trace.

### No-cloning theorem
- **Claim:** There is no unitary $U$ and fixed blank $|\omega\rangle$ such that $U|\psi\rangle|\omega\rangle = |\psi\rangle|\psi\rangle$ for **all** $|\psi\rangle.$
- **Proof 1 (linearity):** If cloning works for $|0\rangle,|1\rangle,$ linearity would demand  
  $U(\alpha|0\rangle+\beta|1\rangle)|\omega\rangle = \alpha|0\rangle|0\rangle + \beta|1\rangle|1\rangle,$  
  but perfect cloning requires $(\alpha|0\rangle+\beta|1\rangle)^{\otimes 2},$ which has cross terms $ \alpha\beta(|0\rangle|1\rangle + |1\rangle|0\rangle).$ Contradiction.
- **Proof 2 (inner product preservation):** Unitaries preserve inner products. If cloning worked for $|\psi\rangle,|\phi\rangle,$ then  
  $\langle \psi|\phi\rangle = \langle \psi|\phi\rangle^2$ ⇒ $\langle \psi|\phi\rangle \in \{0,1\}.$ So only identical or orthogonal states can be cloned.

### Uncertainty intuition (position vs momentum)
- Collapsing to a position-like eigenstate broadens momentum distribution (and vice versa).



## 2. Worked examples
Below we implement small utilities to:
1. Compute post-measurement states and probabilities for single- and two-qubit systems.
2. Numerically illustrate the no-cloning contradiction.

In [ ]:
# Code: utilities

import numpy as np

# Basic ket helpers
zero = np.array([[1.0],[0.0]])
one  = np.array([[0.0],[1.0]])

def ket(v):
    v = np.array(v, dtype=complex).reshape(-1,1)
    # normalise if not zero
    n = np.linalg.norm(v)
    return v / n if n>0 else v

def kron(*args):
    M = np.array([[1.0]])
    for A in args:
        M = np.kron(M, A)
    return M

# Projector for |v>
def projector(v):
    return v @ v.conj().T

# Measurement on a state vector with projectors {Pi_i}
def measure_state(psi, projectors):
    probs = [float((psi.conj().T @ P @ psi).real) for P in projectors]
    outcomes = []
    for i,P in enumerate(projectors):
        p = probs[i]
        if p > 0:
            post = P @ psi / np.sqrt(p)
        else:
            post = None
        outcomes.append((i, p, post))
    return outcomes

# Partial measurement on first qubit in Z basis for a 2-qubit pure state
Z0 = projector(zero)
Z1 = projector(one)

def measure_first_qubit_Z(psi2):
    P0 = kron(Z0, np.eye(2))
    P1 = kron(Z1, np.eye(2))
    probs = []
    posts = []
    for P in (P0,P1):
        num = P @ psi2
        p = float((num.conj().T @ num).real)
        probs.append(p)
        if p>0:
            posts.append(num/np.sqrt(p))
        else:
            posts.append(None)
    return probs, posts

# Pretty print vector (2^n dim)
def show_state(psi, tol=1e-10):
    psi = psi.flatten()
    terms = []
    n = int(np.log2(len(psi)))
    for i,amp in enumerate(psi):
        if abs(amp) > tol:
            b = format(i, f"0{n}b")
            terms.append(f"{amp:.3g}|{b}>")
    return " + ".join(terms) if terms else "0"


In [ ]:
# Example: partial measurement two-qubit

# Example: two-qubit state and partial measurement on qubit A
alpha, beta, gamma, delta = 1/np.sqrt(10), 2/np.sqrt(10), 1/np.sqrt(10), 2/np.sqrt(10)
psi2 = alpha*kron(zero,zero) + beta*kron(zero,one) + gamma*kron(one,zero) + delta*kron(one,one)
print("Initial |ψ> =", show_state(psi2))

probs, posts = measure_first_qubit_Z(psi2)
for outcome, (p, post) in enumerate(zip(probs, posts)):
    print(f"Outcome A={outcome} with p={p:.3f}:  |ψ'> =", show_state(post) if post is not None else "—")


Initial |ψ> = 0.316|00> + 0.632|01> + 0.316|10> + 0.632|11>
Outcome A=0 with p=0.500:  |ψ'> = 0.447|00> + 0.894|01>
Outcome A=1 with p=0.500:  |ψ'> = 0.447|10> + 0.894|11>


/tmp/ipython-input-2443067568.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p = float((num.conj().T @ num).real)


In [ ]:
# Example: single-qubit measurement in arbitrary basis

# Define an arbitrary orthonormal basis {|u>, |u_perp>} and measure
theta, phi = 0.37, 1.21
u = ket([np.cos(theta/2), np.exp(1j*phi)*np.sin(theta/2)])
u_perp = ket([-np.exp(-1j*phi)*np.sin(theta/2), np.cos(theta/2)])

psi = ket([0.6, 0.8j])  # arbitrary state
outs = measure_state(psi, [projector(u), projector(u_perp)])
for i, p, post in outs:
    print(f"Outcome {i}, p={p:.4f}, |ψ'> =", show_state(post))


Outcome 0, p=0.5319, |ψ'> = 0.98+0.07j|0> + 0.0525+0.176j|1>
Outcome 1, p=0.4681, |ψ'> = -0.168-0.0746j|0> + -0.056+0.981j|1>


/tmp/ipython-input-3639086099.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  probs = [float((psi.conj().T @ P @ psi).real) for P in projectors]


In [ ]:
# No-cloning: numerical illustration

# Suppose a "cloner" is linear and clones |0>, |1> with a fixed blank |ω>=|0>
# Define U's action on basis inputs (not actually building a full unitary here):
def putative_clone(psi):
    # emulate linear extension: clone basis but superpositions map linearly
    # U|0>|0> -> |0>|0>, U|1>|0> -> |1>|1>
    a, b = psi.flatten()
    out = a * kron(zero, zero) + b * kron(one, one)
    return out

# Now test superposition |ψ> = (|0>+|1>)/√2
psi = ket([1/np.sqrt(2), 1/np.sqrt(2)])
lhs = putative_clone(psi)                 # linear output
rhs = ket([1/np.sqrt(2), 1/np.sqrt(2)])   # desired single
rhs = kron(rhs, rhs)                      # desired perfect clone |ψ>⊗|ψ>

print("Linear extension output:   ", show_state(lhs))
print("Perfect-clone target:      ", show_state(rhs))
print("Are they equal (within tol)?", np.allclose(lhs, rhs, atol=1e-9))


Linear extension output:    0.707+0j|00> + 0.707+0j|11>
Perfect-clone target:       0.5+0j|00> + 0.5+0j|01> + 0.5+0j|10> + 0.5+0j|11>
Are they equal (within tol)? False


# Key equations

## 3. Key equations and statements

- **Born rule:** $p(m) = \langle\psi|\Pi_m|\psi\rangle.$
- **Post-measurement state (Lüders):** $|\psi'\rangle = \Pi_m|\psi\rangle/\sqrt{p(m)}.$
- **No-cloning (inner-product proof):** If $U|\psi\rangle|\omega\rangle = |\psi\rangle|\psi\rangle$ and $U|\phi\rangle|\omega\rangle = |\phi\rangle|\phi\rangle$ for all $|\psi\rangle,|\phi\rangle,$ then  
  $ \langle \psi|\phi\rangle = \langle \psi|\phi\rangle^2 \Rightarrow \langle \psi|\phi\rangle \in \{0,1\}.$
- **Linearity contradiction:** Cloning $|0\rangle,|1\rangle$ implies $U(\alpha|0\rangle+\beta|1\rangle)|\omega\rangle$ lacks the cross terms required by $(\alpha|0\rangle+\beta|1\rangle)^{\otimes 2}.$



# Try it yourself

## 4. Try it yourself

1. **Partial measurement practice**  
   Modify the two-qubit state in the example and verify the post-measurement states for different bases (e.g., X or Y on the first qubit).

2. **Custom projector**  
   Build a function that measures in the $|\pm\rangle$ basis and confirm that repeated measurement yields deterministic outcomes.

3. **Cloning check (inner-product)**  
   Generate random single-qubit states $|\psi\rangle,|\phi\rangle$ and compute $s=\langle\psi|\phi\rangle.$ Show numerically that demanding cloning would require $s=s^2.$


## 5. Reflection

- What clicked for you about projectors and renormalisation today?  
- Where do you still feel fuzzy (partial vs full measurement, or the logic of the no-cloning proofs)?  
- One sentence you would teach your future self about Day 12:


---
### Links
- **Open in Colab (from GitHub):** replace `YOUR_GITHUB_USERNAME/qucode-21days`
  - `https://colab.research.google.com/github/YOUR_GITHUB_USERNAME/qucode-21days/blob/main/Day12_Bernstein-Vazirani_Algorithm.ipynb.ipynb`
- **Report an issue / suggest a fix:** link to your repo issues page
